In [11]:
import os
import pandas as pd

In [12]:
os.chdir('/Volumes/USB30FD/LA-HEALTH-COVID-Data/data_10-03-2020/')
RCAC_oct0220_df = pd.read_csv('Residual_Congregate_and_Acute_Care_Settings.csv')

In [13]:
os.chdir('/Volumes/USB30FD/LA-HEALTH-COVID-Data/data_10-02-2020/')
RCAC_oct0120_df = pd.read_csv('Residual_Congregate_and_Acute_Care_Settings.csv')

In [16]:
RCAC_oct0120_df.shape

(205, 5)

In [30]:
# RCAC_oct0120_df.groupby('location_name').count().sort_values('city_name', ascending=False)
len(RCAC_oct0120_df['location_name'].unique())

205

In [17]:
RCAC_oct0120_df.head()

,location_name,city_name,number_of_confirmed_staff,number_of_confirmed_residents,total_deaths
0,A and M Armour Home Care,"Pomona, CA",7,4,0
1,AAA Royal Senior Living,"Tarzana, CA",0,1,1
2,Affinity Healthcare Center,"Paramount, CA",6,6,0
3,Alameda Care Center,"Burbank, CA",2,4,0
4,Alcott Rehabilitation Hospital,"Los Angeles, CA",10,22,3


In [31]:
RCAC_oct0220_df.shape

(197, 5)

In [33]:
# RCAC_oct0220_df.groupby('location_name').count().sort_values('city_name', ascending=False)
len(RCAC_oct0220_df['location_name'].unique())


197

In [35]:
oct1_oct2_df = pd.merge(RCAC_oct0120_df, RCAC_oct0220_df, on=['location_name', 'city_name'], how='inner')


In [36]:
oct1_oct2_df.head()

,location_name,city_name,number_of_confirmed_staff_x,number_of_confirmed_residents_x,total_deaths_x,number_of_confirmed_staff_y,number_of_confirmed_residents_y,total_deaths_y
0,A and M Armour Home Care,"Pomona, CA",7,4,0,7,4,0
1,Affinity Healthcare Center,"Paramount, CA",6,6,0,6,6,0
2,Alameda Care Center,"Burbank, CA",2,4,0,2,4,0
3,Alcott Rehabilitation Hospital,"Los Angeles, CA",10,22,3,10,22,3
4,"Alhambra Healthcare & Wellness Centre, LP","Alhambra, CA",1,3,0,1,3,0
